<h3>Ejemplo de diseño de un emprendimiento</h3>

_G. Raush, MF-UPC, 2019_

En un lugar se necesita disponer de una potencia eléctrica de 5800 CV. Para ello se desea aprovechar un salto de agua con una altura de salto bruto de 75 m. Con el propósito de conseguirlo se desea utilizar una turbina hidráulica con un rendimiento del 75%, acoplada a un genrador con un rendimiento de 80%.

Se desea conocer:
- el caudal de agua necesario
- el rendimeinto total de la instalación
- la energía producida por el salto en el tiempo de un año, si el caudal se mantiene constante
- una estimación del volumen y área de reservorio de la presa para garantizar esa producción continuada 

<i>Observaciones</i>: Considerar un generados con $Z = 8$ pares de polos para una red de 50 Hz.


In [27]:
from math import *
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.constants import g, pi

In [28]:
area = lambda x : 0.25 * pi * x**2
reynolds = lambda q,d : 4 * q / pi / d *1e6
fMoody = lambda e,d,re : 0.25 / (log10(e / d / 3.7 + 5.74 / pow(re,0.9) ))**2

In [29]:
rho,Q,Hprima,etaTurb,etaGen = sp.symbols('rho,Q,Hprima,etaTurb,etaGen')

<b>Rendimiento del salto</b>

$$\eta_{turb} = \frac{H^{''}}{H^{'}}$$

- $H^{'}$ : altura de salto bruto, m
- $H^{''}$ : altura de salto neto, m

In [30]:
Hbrute = 75 # m
etaTurb = 0.75 
Hprima = Hbrute * etaTurb
print('Hprima = %.2f m' % Hprima)

Hprima = 56.25 m


In [31]:
rho = 1000 # kg/m^3

In [32]:
etaGen = 0.80
PowElec = 5800 # CV
Eq1 = sp.Eq(PowElec, rho * g * Hprima * Q * etaGen / 746)

In [33]:
Eq1

Eq(5800, 591.553954423592*Q)

In [34]:
Qnum = sp.solve(Eq1, Q)[0] #.solve(Q)
print("Caudal turbinado Q = %.1f m^3/s" % Qnum)

Caudal turbinado Q = 9.8 m^3/s


<b>Diámetro Óptimo</b>

$$D_{o} = \sqrt[7]{\frac{26.67 C_E  N Q^{3} \eta  f \sigma_S}{g \pi^{2} \rho_S C_S T_Y H^{'}  }}$$

- $C_E$ : Costo del la enegía en €/W-h
- $C_S$ : Costo del acero en €/kg
- $N$ : horas anuales, 8760 hr/año
- $Q$ : caudal turbinado en $m^3/s$
- $H^{'}$ : Altura del salto bruto 
- $\eta$ : rendimiento de la instalación
- $f$ : factor de fricción de Moody
- $\rho_S$ : densidad del acero, 7800 $kg/m^3$
- $\sigma_S$ : tensión de ruptura del acero, 80 MPa 
- $T_Y$ : Tasa anual de retorno de la inversión


In [35]:
Q = Qnum
N = 365*24
CostoEnergia = 0.01/1000 # kW*hr
CostoAcero = 1  # Euros/kg
TasaAnualRetorno = 0.05
rho = 1000
rhoAcero = 7800 # kg/m^3
etaGrupo = 1
strenghtSteel = 80e6 # MPa
f = 0.01 # 0.025
avergPress = (Hbrute * rho * g ) * 1.5 # presion media contemplando la posibilidad del golpe de ariete
numer = 40 * rho * f * Q**3 * etaGrupo * N * CostoEnergia * strenghtSteel
denom = pi**2 * avergPress * rhoAcero * CostoAcero * TasaAnualRetorno
ratio = numer / denom
D = pow(ratio,1./7.)
print("D = %.3f m" % D)
   

D = 2.507 m


In [36]:
V = Qnum / area(D)
print("V = %.1f m/s" % V)

V = 2.0 m/s


In [37]:
print("Re = %0.f" %reynolds(Qnum,D))

Re = 4979809


In [38]:
print("f = %.3f" % fMoody(0.1,D*1e3,4e6))

f = 0.011


**Potencia hidráulica total** $$P_{tot} = \frac{\rho g H^{'} Q}{746}$$ en CV 

In [39]:
PowTot = rho * g * Qnum * Hbrute / 746
print("Pot Total = %.1f CV" % PowTot)

Pot Total = 9666.7 CV


**Rendimiento Total de la Instalación** $$\eta = \frac{P_e}{P_t}$$

In [40]:
eta = PowElec / PowTot
print("eta = %.1f %%" % (eta*1e2))

eta = 60.0 %


<b>Energía anual generada</b>

$$W_e = N P_e$$

In [41]:
N = 365 * 24 # horas anuales
WElec = N * PowElec / 1.32
print("Energía anual = %.0f kW-h" % WElec)

Energía anual = 38490909 kW-h


<b>Volumen de Agua turbinado</b>

$$V = 3600  N  Q$$

In [42]:
Vol = 3600 * N * Qnum
print("V = %.2e km³/año" % (Vol*(1e-3)**3))

V = 3.09e-01 km³/año


In [43]:
# Profundidad media de un embalse
hMed = 15 # m
AreaMedia = Vol / hMed
print("Area media = %.0f km²" % (AreaMedia * (1e-3)**2))

Area media = 21 km²


<b>Tipo de máquina</b>

Frecuencia de red 50 Hz, número de polos del generador $Z = 8$

In [44]:
fHz = 50 # Hz
Z = 8
Ngen = 60 * fHz / Z
print("N gen = %.1f rpm" % Ngen)

N gen = 375.0 rpm


Velocidad específica de $n_s = N_{gen} \frac{\sqrt{P}}{H \sqrt[4]{H}}$ 

- $P$ en CV
- $H$ en m
- $N_{gen}$ en rpm

In [45]:
Ns = Ngen * sqrt(PowTot) / Hprima / Hprima**0.25

In [46]:
print("Ns = %.0f rpm" % Ns)

Ns = 239 rpm


La turbina adecuada es una con una altura de aspiración de 3 m

![grafico_Turbinas](./graficos-Turb_Veloc-Especif-EjemploDesign.png)